# Pythonを用いた競馬予測

## インポート

In [47]:
import re  # 正規表現
import time
from urllib.request import Request, urlopen

import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

## データ収集
- カレンダーのページから開催日一覧を取得（スクレイピング）
    1. 2023年1月：https://race.netkeiba.com/top/calendar.html?year=2023&month=1
- 開催ページからレースid一覧を取得
    1. 2023年1月5日開催：https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105
- レース結果ページからレース結果テーブル一覧を取得

In [3]:
from urllib.request import Request, urlopen

url = "https://db.netkeiba.com/race/202306050811/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()
print(html)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xcd\xad\xc7\xcf\xb5\xad\xc7\xb0\xa1\xc32023\xc7\xaf12\xb7\xee24\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1

In [11]:
import pandas as pd

pd.read_html(html)

NameError: name 'html' is not defined

In [5]:
pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,3,5,ドウデュース,牡4,58,武豊,2:30.9,NaN,5.2,2,506(+2),[西] 友道康夫
1,2,8,16,スターズオンアース,牝4,56,ルメール,2:31.0,1/2,8.6,7,494(-2),[東] 高柳瑞樹
2,3,2,4,タイトルホルダー,牡5,58,横山和生,2:31.2,1,8.3,6,474(-2),[東] 栗田徹
3,4,5,10,ジャスティンパレス,牡4,58,横山武史,2:31.2,アタマ,3.6,1,474(+6),[西] 杉山晴紀
4,5,1,2,シャフリヤール,牡5,58,松山弘平,2:31.2,クビ,44.6,8,454(0),[西] 藤原英昭
5,6,7,13,タスティエーラ,牡3,56,ムーア,2:31.5,1.1/2,7.1,5,498(+18),[東] 堀宣行
6,7,6,12,ウインマリリン,牝6,56,モリス,2:31.6,3/4,171.9,14,476(0),[東] 手塚貴久
7,8,1,1,ソールオリエンス,牡3,56,川田将雅,2:31.6,アタマ,6.5,4,466(+2),[東] 手塚貴久
8,9,6,11,ハーパー,牝3,54,岩田望来,2:31.6,クビ,93.2,11,474(-10),[西] 友道康夫
9,10,2,3,ホウオウエミーズ,牝6,56,田辺裕信,2:31.7,クビ,193.2,16,452(+2),[東] 池上昌和


In [6]:
pd.read_html(html)[1]

,0,1,2,3
0,単勝,5,520,2
1,複勝,5 16 4,230 240 330,3 5 7
2,枠連,3 - 8,980,2
3,馬連,5 - 16,2730,15


In [7]:
pd.read_html(html)[2]

,0,1,2,3
0,ワイド,5 - 16 4 - 5 4 - 16,"930 1,210 1,720",13 17 21
1,馬単,5 → 16,4380,23
2,三連複,4 - 5 - 16,8050,28
3,三連単,5 → 16 → 4,42110,159


### 開催日一覧を取得する

In [50]:
url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()
html

HTTPError: HTTP Error 400: Bad Request

In [9]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_common_race_schedule (d) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>開催日程 | レース情報(JRA) - netkeiba</title>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,開催日程,スケジュール,開催日,netkeiba,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba_logo_keiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="開催日程 | レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/top/calendar.html" property="og:url"/>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最

In [10]:
a = soup.find("table", class_="Calendar_Table").find("a")
a

<a href="../top/race_list.html?kaisai_date=20230105" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">5</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">中京</span><span class="JName">京都金杯</span></p>
</div><!-- /. RaceKaisaiBox-->
</a>

In [11]:
a["href"]

'../top/race_list.html?kaisai_date=20230105'

In [12]:
import re # 正規表現

re.findall(r"kaisai_date=(\d{8})", a["href"])[0] # \d{8}：8桁の数字（\dは数字）()を付けるとその部分だけ取り出される

'20230105'

In [13]:
a_list = soup.find("table", class_="Calendar_Table").findAll("a")

In [14]:
kaisai_date_list = []
for a in a_list:
    kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
    kaisai_date_list.append(kaisai_date)

kaisai_date_list

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129']

In [15]:
import time
from tqdm.notebook import tqdm

def scrape_kaisai_date(from_, to_):
    
    kaisai_date_list = []
    
    for date in tqdm(pd.date_range(from_, to_, freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
        req = Request(url, headers=headers)
        html = urlopen(req).read()
        
        time.sleep(1)
        
        soup = BeautifulSoup(html, 'html.parser')
        
        a_list = soup.find("table", class_="Calendar_Table").find_all("a")
        
        for a in a_list:
            kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_date_list.append(kaisai_date)
        
    return kaisai_date_list

In [16]:
scrape_kaisai_date("2023-01", "2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 開催ページからrace_idを取得

In [54]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"

In [55]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [56]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
req = Request(url, headers=headers)
html = urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')


In [57]:
soup.find("div", class_="RaceList_Box") # 動的jsで構成されている場合はBeautifulSoupでは要素を取得できない.

取得できないので、ChromeDriverを使う

In [58]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

driver_path = ChromeDriverManager().install()
driver_path


'C:\\Users\\go200\\.wdm\\drivers\\chromedriver\\win64\\131.0.6778.204\\chromedriver-win32/chromedriver.exe'

In [59]:
driver = webdriver.Chrome(service=Service(driver_path))

In [60]:
driver.get(url)

In [61]:
from selenium.webdriver.common.by import By
li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.67")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.68")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.69")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.70")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.71")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a8

In [62]:
li = li_list[0]
li

<selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.67")>

In [63]:
href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
href

'https://race.netkeiba.com/race/result.html?race_id=202306010101&rf=race_list'

In [64]:
import re

re.findall(r"race_id=(\d{12})", href)[0]

'202306010101'

In [65]:
race_id_list = []

for li in li_list:
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    race_id_list.append(race_id)

In [66]:
len(race_id_list)

24

In [67]:
driver.quit()

## 関数化

In [68]:
import scraping

In [69]:
kaisai_date_list = scraping.scrape_kaisai_date(from_="2023-01", to_="2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [70]:
import time
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
import traceback

def scrape_race_id_list(kaisai_date_list: list[str]):
    options = Options()
    options.add_argument("--headless") # 処理軽量化のためにバックグラウンドで実行
    driver_path = ChromeDriverManager().install()
    race_id_list = []
    
    # for文終了時にwith構文自動的にdriverがquitする.
    with webdriver.Chrome(service=Service(driver_path),options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc()) # エラー把握
                break
    return race_id_list

In [71]:
race_id_list = scrape_race_id_list(kaisai_date_list)

  0%|          | 0/107 [00:00<?, ?it/s]

stopped at https://race.netkeiba.com/top/race_list.html?kaisai_date=20230107
Traceback (most recent call last):
  File "C:\Users\go200\AppData\Local\Temp\ipykernel_30360\893257457.py", line 18, in scrape_race_id_list
    time.sleep(1)
KeyboardInterrupt



In [72]:
race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112']

### スクリプトのチェック

In [35]:
import scraping
%load_ext autoreload

In [36]:
%autoreload

In [ ]:
race_id_list = scraping.scrape_race_id_list(kaisai_date_list[:10])

notebookにおけるモジュールのリロード

In [73]:
race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112']

In [77]:
from urllib.request import urlopen

url = "https://db.netkeiba.com/race/202306050811/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()

In [ ]:
import pandas as pd

pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,3,5,ドウデュース,牡4,58,武豊,2:30.9,NaN,5.2,2,506(+2),[西] 友道康夫
1,2,8,16,スターズオンアース,牝4,56,ルメール,2:31.0,1/2,8.6,7,494(-2),[東] 高柳瑞樹
2,3,2,4,タイトルホルダー,牡5,58,横山和生,2:31.2,1,8.3,6,474(-2),[東] 栗田徹
3,4,5,10,ジャスティンパレス,牡4,58,横山武史,2:31.2,アタマ,3.6,1,474(+6),[西] 杉山晴紀
4,5,1,2,シャフリヤール,牡5,58,松山弘平,2:31.2,クビ,44.6,8,454(0),[西] 藤原英昭
5,6,7,13,タスティエーラ,牡3,56,ムーア,2:31.5,1.1/2,7.1,5,498(+18),[東] 堀宣行
6,7,6,12,ウインマリリン,牝6,56,モリス,2:31.6,3/4,171.9,14,476(0),[東] 手塚貴久
7,8,1,1,ソールオリエンス,牡3,56,川田将雅,2:31.6,アタマ,6.5,4,466(+2),[東] 手塚貴久
8,9,6,11,ハーパー,牝3,54,岩田望来,2:31.6,クビ,93.2,11,474(-10),[西] 友道康夫
9,10,2,3,ホウオウエミーズ,牝6,56,田辺裕信,2:31.7,クビ,193.2,16,452(+2),[東] 池上昌和


In [80]:
import pickle

with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [82]:
len(race_id_list)

3456

In [83]:
for race_id in race_id_list:
    url = f"https://db.netkeiba.com/race/{race_id}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

    req = Request(url, headers=headers)
    html = urlopen(req).read()
    
    pd.read_html(html)[0]

In [84]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xa5\xab\xa5\xa6\xa5\xf3\xa5\xc8\xa5\xc0\xa5\xa6\xa5\xf3S\xa1\xc32023\xc7\xaf12\xb7\xee28\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa

拡張性のためにHTMLを保存する.

In [89]:
with open("../data/html/202306050811.bin", "wb") as f:
    f.write(html)

In [4]:
from pathlib import Path # windowsとmacによる環境の違いを防ぐためにpathlibを使用

HTML_DIR = Path("..", "data", "html")

In [95]:
with open(HTML_DIR / "202306050811.bin", "wb") as f:
    f.write(html)

In [96]:
import time

for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    req = Request(url, headers=headers)
    html = urlopen(req).read()
    time.sleep(1)
    pd.read_html(html)[0]
    
    with open(HTML_DIR / f"{race_id}.bin", "wb") as f:
        f.write(html)

In [135]:
from tqdm.notebook import tqdm
def scrape_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin" # 外で定義した変数を関数内で直接使用しないで, 引数に渡すことを心掛ける.
        # binファイルがすでに存在する場合はスキップする.
        if filepath.is_file():
            print(f"skipped: {race_id}")
        else:
            url = f"https://db.netkeiba.com/race/{race_id}"
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
            req = Request(url, headers=headers)
            html = urlopen(req).read()
            time.sleep(1)
            pd.read_html(html)[0]
            with open(filepath, "wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [140]:
html_path_list = scrape_html_race(race_id_list[:14], save_dir=HTML_DIR)

  0%|          | 0/14 [00:00<?, ?it/s]

skipped: 202306010101
skipped: 202306010102
skipped: 202306010103
skipped: 202306010104
skipped: 202306010105
skipped: 202306010106
skipped: 202306010107
skipped: 202306010108
skipped: 202306010109
skipped: 202306010110
skipped: 202306010111
skipped: 202306010112
skipped: 202307010101
skipped: 202307010102


In [141]:
html_path_list # 新しくスクレイピングしたもののみが帰ってくる.

[]

In [117]:
html_path_list[0].is_file()

True

In [120]:
(HTML_DIR / "aaaa.bin").is_file() # is_fileメソッドで存在するファイルをチェックできる.

False

In [6]:
HTML_RACE_DIR = HTML_DIR / "race"
HTML_RACE_DIR

WindowsPath('../data/html/race')

In [7]:
html_paths_race = list(HTML_RACE_DIR.glob("*")) # glob"*"で全てのPathを取得.
len(html_paths_race)

3456

In [44]:
dfs = {}
for html_path in html_paths_race:
    with open(html_path, "rb") as f:
       race_id = html_path.stem # pathからidのみを取り出せる.
       html = f.read()
       df = pd.read_html(html)[0]
       df.index = [race_id] * len(df) # 難しい：indexを1レースの行数分指定する必要があるので * len(df)
       dfs[race_id] = df
concat_df = pd.concat(dfs.values()) # 複数のテーブルを結合できる.
concat_df.index.name = "race_id"

In [45]:
concat_df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202305010101,1,7,14,メイデンタワー,牝3,54,横山武史,1:26.1,NaN,1.6,1,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54,津村明秀,1:26.8,4,6.0,3,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54,嶋田純次,1:27.1,2,92.7,10,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54,内田博幸,1:27.3,3/4,13.6,4,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52,小林脩斗,1:27.9,3.1/2,5.8,2,450(+2),[東] 奥平雅士
202305010101,6,4,8,スピリチュアル,牝3,54,武藤雅,1:28.0,1/2,25.7,7,444(-6),[東] 武藤善則
202305010101,7,6,11,マラマプア,牝3,53,永野猛蔵,1:28.1,1,15.2,5,482(+2),[東] 伊藤圭三
202305010101,8,2,4,フジジュンフェイス,牝3,54,杉原誠人,1:28.5,2.1/2,108.1,11,442(-4),[東] 伊藤大士
202305010101,9,4,7,テイエムガーネット,牝3,54,武士沢友,1:28.6,1/2,22.6,6,468(+4),[東] 石栗龍彦


In [19]:
from tqdm.notebook import tqdm
def create_resuls(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_paths_race):
        with open(html_path, "rb") as f:
            race_id = html_path.stem # pathからidのみを取り出せる.
            html = f.read()
            df = pd.read_html(html)[0]
            df.index = [race_id] * len(df) # 難しい：indexを1レースの行数分指定する必要があるので * len(df)
            dfs[race_id] = df
    concat_df = pd.concat(dfs.values()) # 複数のテーブルを結合できる.
    concat_df.index.name = "race_id"
    return concat_df

In [20]:
results = create_resuls(html_path_list=html_paths_race)

  0%|          | 0/3456 [00:00<?, ?it/s]

In [21]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN,1.2,1.0,452(-4),[東] 鈴木伸尋
202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ,4.1,2.0,454(+2),[東] 杉浦宏昭
202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2,59.9,6.0,438(-6),[西] 羽月友彦
202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2,16.6,3.0,450(+2),[西] 武幸四郎
202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2,23.9,5.0,434(-10),[西] 今野貞一
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4,62.8,11.0,394(+2),[西] 松永幹夫
202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2,125.6,12.0,470(-4),[西] 安田隆行
202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4,9.8,5.0,496(+8),[西] 森田直行
